In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from fancyimpute import KNN    
from sklearn import preprocessing

from xgboost.sklearn import XGBClassifier, XGBRegressor
import catboost as ctb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
from sklearn import linear_model
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.stats import mode

In [ ]:
from sklearn.ensemble import RandomForestClassifier
  
 # create regressor object 
rf = RandomForestClassifier(n_estimators = 100, random_state = 0) 

In [ ]:
gbm = lgb.LGBMRegressor()
cat = ctb.CatBoostRegressor()
xgb = XGBRegressor()
svr = svm.SVR()

In [ ]:
train_file = "/kaggle/input/a-fine-windy-day-hackerearth-ml-challenge/train_data.csv"
test_file = "/kaggle/input/a-fine-windy-day-hackerearth-ml-challenge/test_data.csv"

In [ ]:
train_df = pd.read_csv(train_file)

In [ ]:
train_df.head()

In [ ]:
train_df.isna().sum()

In [ ]:
days = (pd.datetime.now() - pd.to_datetime(train_df['datetime'])).dt.days
train_df.insert(loc=2, column='Days', value=days)

In [ ]:
train_df.shape

In [ ]:
def turbine_status_to_numeric(x):
    if x=='AAA': return 0
    if x=='A':   return 1
    if x=='BD': return 2
    if x=='B2': return 0
    if x=='ABC':   return 1
    if x=='AC': return 2
    if x=='BCD': return 0
    if x=='BA':   return 1
    if x=='A2': return 2
    if x=='AB': return 0
    if x=='BBB':   return 1
    if x=='B': return 2
    if x=='D': return 0
    if x=='BB':   return 1
    else : return np.nan
    
def cloud_level_to_numeric(x):
    if x=='Extremely Low': return 0
    if x=='Low':   return 1
    if x=='Medium':   return 2
    else : return np.nan


In [ ]:
train_df["turbine_status"] = train_df["turbine_status"].apply(turbine_status_to_numeric)
train_df["cloud_level"] = train_df["cloud_level"].apply(cloud_level_to_numeric)

In [ ]:
df = train_df.copy()
# train_df = None

In [ ]:
delete_columns = ["tracking_id","datetime"]
df = df.drop(delete_columns, 1)

In [ ]:
df.corr()['windmill_generated_power(kW/h)'].sort_values()

In [ ]:
df = df[df['atmospheric_pressure(Pascal)'].notna()]

In [ ]:
knn = KNN(k=3)

In [ ]:
# X is the complete data matrix
# X_incomplete has the same values as X except a subset have been replace with NaN

# Use 3 nearest rows which have a feature to fill in each row's missing features
X_new = knn.fit_transform(df)

In [ ]:
df_new = pd.DataFrame(X_new)

In [ ]:
df_new.head()

In [ ]:
# fill_mode = lambda col: col.fillna(col.mode())
# train.apply(fill_mode, axis=0)

In [ ]:
X = df_new.iloc[:,:-1]
y = df_new.iloc[:,-1]

In [ ]:
y = np.array(y)
X = np.array(X)

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# svc.fit(X_train, y_train)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
cat.fit(X_train, y_train)

In [ ]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)])

In [ ]:
# svm.SVR()
# linear_model.BayesianRidge()

In [ ]:
gbm_pred = gbm.predict(X_test)
cat_pred = cat.predict(X_test)
xgb_pred = xgb.predict(X_test)
# rf_pred = rf.predict(X_test)

In [ ]:
def get_mode(a,b,c):
    return mode([a,b,c])[0][0]

In [ ]:
# y_pred = np.average(gbm_pred,cat_pred,xgb_pred)
y_pred = [(a+b+c)/3 for a, b, c in zip(xgb_pred,cat_pred,gbm_pred)]

In [ ]:
from sklearn import metrics

In [ ]:
score = max(0, 100*metrics.r2_score(y_test, y_pred))
score

In [ ]:
test_df = pd.read_csv(test_file)

In [ ]:
test_df.head()

In [ ]:
days = (pd.datetime.now() - pd.to_datetime(test_df['datetime'])).dt.days
test_df.insert(loc=2, column='Days', value=days)

In [ ]:
ids = test_df["tracking_id"]
dates = test_df["datetime"]

In [ ]:
test_df.columns

In [ ]:
test = test_df.drop(delete_columns, 1)

In [ ]:
test.isna().sum()

In [ ]:
test["turbine_status"] = test["turbine_status"].apply(turbine_status_to_numeric)
test["cloud_level"] = test["cloud_level"].apply(cloud_level_to_numeric)

In [ ]:
val_X = test.iloc[:,:]

In [ ]:
val_X = np.array(val_X)
val_X.shape

In [ ]:
# val_X_knn = knn.fit_transform(val_X)

In [ ]:
# val_X_selected = fs.transform(val_X_knn)

In [ ]:
gbm_pred = gbm.predict(val_X, num_iteration=gbm.best_iteration_)
cat_pred = cat.predict(val_X)
xgb_pred = xgb.predict(val_X)
# rf_pred = rs.predict(val_X_selected)

In [ ]:
predictions = [(a+b+c)/3 for a, b, c in zip(xgb_pred,cat_pred,gbm_pred)]

In [ ]:
result = pd.DataFrame()
result["tracking_id"] = ids
result["datetime"] = dates
result["windmill_generated_power(kW/h)"] = predictions
result.to_csv("/kaggle/working/results.csv",index=False)

In [ ]:
# https://www.kaggle.com/lasmith/house-price-regression-with-lightgbm

In [ ]:
result.head(100)